V:2021/10/08

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
%matplotlib inline

In [2]:
year="2015"
month="1503"
pathg='S_N_DF/'           # Have to be specified      

In [4]:
divi=6
nwindow=2**13
fm=256
nintervd=divi*24
df=float(fm)/float(nwindow)
bandcal=np.array([6,25],dtype=float)
nhour=fm*3600
fre=np.arange(bandcal[0],bandcal[1]+df,df)   # calibrated frequec
nf=len(fre)
def pofre(f,df,fi):
    return int(round((f-fi)/df))

In [5]:
pathmonth=pathg+year+"/"+month+"/"+"SR"+month[0:4]+"_"

fichmedia0=pathmonth+"media"+"_0"
fichmedia1=pathmonth+"media"+"_1"

fichsatper0=pathmonth+"satper"+"_0"
fichsatper1=pathmonth+"satper"+"_1"                               
                               
medidas0=np.genfromtxt(fichmedia0).reshape(-1,nf)
medidas1=np.genfromtxt(fichmedia1).reshape(-1,nf)
satper0=np.genfromtxt(fichsatper0)
satper1=np.genfromtxt(fichsatper1)
nintervm=len(medidas0)
nhours=nintervm//divi
ndays=nhours//24

In [6]:
def spectrumNoisy(i,nho,ndi):
    indi=i+nho*6+ndi*24
    plt.grid()
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'Amplitude pT/$\sqrt{Hz}$')
    plt.title('Amplitude spectrum')
    plt.plot(fre,medidas0[indi,:],c='b',label='N-S sensor')
    plt.plot(fre,medidas1[indi,:],c='r',label='E-W sensor')
    plt.legend(loc=1)
    plt.show()

In [7]:
interact(spectrumNoisy,i=(0,6-1),nho=(0,23),ndi=(0,ndays-1))

interactive(children=(IntSlider(value=2, description='i', max=5), IntSlider(value=11, description='nho', max=2…

<function __main__.spectrumNoisy(i, nho, ndi)>

In [8]:
def recta(val,frec):
    a=(val[-1]-val[0])/(frec[-1]-frec[0])
    b=val[0]
    valmod=a*(frec-frec[0])+b
    de=np.max((val-valmod)/valmod)+np.min((val-valmod)/valmod)
    pmax=np.where(val==np.max(val))[0][0]
    return valmod, de, [frec[pmax],val[pmax],valmod[pmax]]

def antro(medidasantro,fre,refpos,antropos,satper,facfil):
    antrolis=[]
    medidas=np.copy(medidasantro)
    for i in range(len(medidas)):
        if satper[i]<0.9999:
            cont=0
            for j in range(len(antropos)):
                desvia = recta(medidas[i,refpos[j,0]:refpos[j,1]+1],\
                               fre[refpos[j,0]:refpos[j,1]+1])[1]
                resul = recta(medidas[i,antropos[j,0]:antropos[j,1]+1],\
                              fre[antropos[j,0]:antropos[j,1]+1])
                if(resul[1]>facfil*np.abs(desvia)):
                    cont=cont+1
                    medidas[i,antropos[j,0]:antropos[j,1]+1]=resul[0]
                    antrolis.append([i,*resul[2]])
                if(cont==2 and j==1):
                    resul = recta(medidas[i,antropos[0,0]:antropos[1,1]+1],\
                                  fre[antropos[0,0]:antropos[1,1]+1])
                    medidas[i,antropos[0,0]:antropos[1,1]+1]=resul[0]
        else:
            print("Saturated interval ",i)  
    return medidas, antrolis

In [9]:
listafrecantro=np.array([[[14.55,14.90],[14.1,14.5]],\
                         [[14.90,15.35],[15.5,15.8]],\
                         [[16.50,16.90],[15.8,16.4]]])
positions=np.array([pofre(f,df,bandcal[0])\
                   for f in listafrecantro.ravel()],dtype=int)
antropos=(positions.reshape(listafrecantro.shape))[:,0]
refepos=(positions.reshape(listafrecantro.shape))[:,1]

In [10]:
medcor0,antrolis0 = antro(medidas0,fre,refepos,antropos,satper0,2.5)
medcor1,antrolis1 = antro(medidas1,fre,refepos,antropos,satper1,2.5)
varsal=1

In [11]:
def spectrumDeNoisy(i,nho,ndi):
    indi=i+nho*6+ndi*24
    plt.grid()
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'Amplitude pT/$\sqrt{Hz}$')
    plt.title('Amplitude spectrum')
    plt.plot(fre,medcor0[indi,:],c='b',label='N-S sensor')
    plt.plot(fre,medcor1[indi,:],c='r',label='E-W sensor')
    plt.legend(loc=1)
    plt.show()

In [12]:
interact(spectrumDeNoisy,i=(0,6-1),nho=(0,23),ndi=(0,ndays-1))

interactive(children=(IntSlider(value=2, description='i', max=5), IntSlider(value=11, description='nho', max=2…

<function __main__.spectrumDeNoisy(i, nho, ndi)>

In [13]:
listafrecantroRe=np.array([[[14.55,15.35],[13.5,13.9]]])
positionsRe=np.array([pofre(f,df,bandcal[0])\
                   for f in listafrecantroRe.ravel()],dtype=int)
antroposRe=(positionsRe.reshape(listafrecantroRe.shape))[:,0]
refeposRe=(positionsRe.reshape(listafrecantroRe.shape))[:,1]

In [14]:
medcor0Re,antrolis0Re = antro(medcor0,fre,refeposRe,antroposRe,satper0,2)
medcor1Re,antrolis1Re = antro(medcor1,fre,refeposRe,antroposRe,satper1,2)
varsal=2

In [15]:
def spectrumDeNoisyAgain(i,nho,ndi):
    indi=i+nho*6+ndi*24
    plt.grid()
    plt.xlabel('Frequency (Hz)')
    plt.ylabel(r'Amplitude pT/$\sqrt{Hz}$')
    plt.title('Amplitude spectrum')
    plt.plot(fre,medcor0Re[indi,:],c='b',label='N-S sensor')
    plt.plot(fre,medcor1Re[indi,:],c='r',label='E-W sensor')
    plt.legend(loc=1)
    plt.show()

In [16]:
interact(spectrumDeNoisyAgain,i=(0,6-1),nho=(0,23),ndi=(0,ndays-1))

interactive(children=(IntSlider(value=2, description='i', max=5), IntSlider(value=11, description='nho', max=2…

<function __main__.spectrumDeNoisyAgain(i, nho, ndi)>

## Outputs

In [17]:
fichsal0=open(pathmonth+"mediaNA"+"_0",mode='w')
fichsal1=open(pathmonth+"mediaNA"+"_1",mode='w')

for i in range(nintervm):
    if varsal==1:
        print(*medcor0[i],sep='\n',file=fichsal0)
        print(*medcor1[i],sep='\n',file=fichsal1)
    elif varsal==2:
        print(*medcor0Re[i],sep='\n',file=fichsal0)
        print(*medcor1Re[i],sep='\n',file=fichsal1)
    else:
        print('Wrong output value')

fichsal0.close()
fichsal1.close()